In [4]:
import os
# Read/write json
import json

In [7]:
# To check a string is an URL or not
from urllib.parse import urlparse
import requests

def scrape_jina_ai(agent_id:str) -> None:
    #1. Read the agent metadata
    agent_dir = os.path.join(os.getcwd(), 'agent_data', agent_id)
    with open(os.path.join(agent_dir, 'metadata.json'), 'r') as file:
        metadata = json.load(file)

    # Read url
    article_url = metadata['url']

    #2. Check the passed argument for an URL or not
    result = urlparse(article_url)
    assert all([result.scheme, result.netloc]), f'{article_url} is not a valid URL'
           
    #3. The filename to save the LLM friendly format
    llm_input_file = f'{agent_dir}/llm_input.text'
    if os.path.isfile(llm_input_file):
        print(f'Skip {llm_input_file} as it exists')
    else:
        response = requests.get('https://r.jina.ai/' + article_url)
        with open(llm_input_file, 'w') as file:
            file.write(response.text)    
        print(f'Written LLM friendly output to {llm_input_file}')

In [14]:
scrape_jina_ai(agent_id='us_biotech')

Skip /home/sugath/py_envs/jupyter/ai/stocks/agent_data/us_biotech/llm_input.text as it exists


In [9]:
# To load modules
import importlib

# AI agent modules
from phi.agent import Agent, RunResponse
from phi.model.groq import Groq

# To read environment property file
from dotenv import load_dotenv

def run_agent(agent_id:str) -> RunResponse:
    #1. Load the response model class dynamically
    module = importlib.import_module(f'agent_data.{agent_id}.model')
    # Retrieve the class from the imported module
    model = getattr(module, 'StockList')

    #2. Agent directory
    agent_dir = os.path.join('agent_data', agent_id)
    # Place to save the response
    response_file = os.path.join(agent_dir, 'response.json')
    
    #3. Read the agent metadata
    with open(f'{agent_dir}/metadata.json', 'r') as file:
        metadata = json.load(file)

    #4. Construct the agent
    agent = Agent(
        name=metadata['name'],
        role=metadata['role'],
        model=Groq(id=metadata['model_id']),
        instructions=metadata['instructions'],
        markdown=True,
        show_tool_calls=True,
        # debug_mode=True,
        save_response_to_file=response_file,
        response_model=model,
        response_format={'type': 'json_object'}
    )
    #5. Load environment variables from .env file
    load_dotenv()
    
    #6. Get the input text to pass to the agent
    input_file = os.path.join(agent_dir, metadata['input'])
    with open(input_file, 'r') as file:
        content = file.read()

    #7. Run the agent and return the response to the caller
    return agent.run(content)

In [17]:
from phi.utils.pprint import pprint_run_response

response:RunResponse = run_agent(agent_id='us_biotech')
pprint_run_response(response)